## Chapter Exercises

### Monad Instances

Write `Monad` instances for the following types. Use the `QuickCheck` properties we showed you to validate your instances.

1.
```haskell
data Nope a = NopeDotJpg 
  deriving (Eq, Show)
```

In [ ]:
data Nope a = NopeDotJpg 
  deriving (Eq, Show)

instance Functor Nope where
  fmap _ NopeDotJpg = NopeDotJpg

instance Applicative Nope where
  pure _ = NopeDotJpg
  (<*>) _ _ = NopeDotJpg

instance Monad Nope where
  return = pure
  (>>=) _ _ = NopeDotJpg

instance Arbitrary (Nope a) where
  arbitrary = return NopeDotJpg

instance EqProp (Nope a) where
  (=-=) = eq

2.
```haskell
data PhhhbbtttEither b a = 
    Lefty a 
  | Righty b 
  deriving (Eq, Show)
```

In [ ]:
data PhhhbbtttEither b a = 
    Lefty a 
  | Righty b 
  deriving (Eq, Show)

instance Functor (PhhhbbtttEither b) where
  fmap f (Lefty a) = Lefty $ f a
  fmap _ (Righty b) = Righty b

instance Applicative (PhhhbbtttEither b) where
  pure = Lefty
  (<*>) (Lefty f) (Lefty a) = Lefty $ f a
  (<*>) (Righty b) _ = Righty b
  (<*>) _ (Righty b) = Righty b

instance Monad (PhhhbbtttEither b) where
  return = pure
  (>>=) (Lefty a) f = f a
  (>>=) (Righty b) _ = Righty b

instance (Arbitrary b, Arbitrary a) => Arbitrary (PhhhbbtttEither b a) where
  arbitrary = do
    x <- arbitrary
    y <- arbitrary
    elements [Lefty x, Righty y]

instance (Eq b, Eq a) => EqProp (PhhhbbtttEither b a) where
  (=-=) = eq

3.
```haskell
newtype Identity a = Identity a 
  deriving (Eq, Ord, Show)
```

In [ ]:
newtype Identity a = Identity a 
  deriving (Eq, Ord, Show)

instance Functor Identity where
  fmap f (Identity a) = Identity $ f a

instance Applicative Identity where
  pure = Identity
  (<*>) (Identity f) (Identity a) = Identity $ f a

instance Monad Identity where
  return = pure
  (>>=) (Identity a) f = f a

instance Arbitrary a => Arbitrary (Identity a) where
  arbitrary = Identity <$> arbitrary

instance Eq a => EqProp (Identity a) where
  (=-=) = eq

4.
```haskell
data List a =
    Nil
  | Cons a (List a)
  deriving (Eq, Show)
```

In [ ]:
instance Functor List where
  fmap :: (a -> b) -> List a -> List b
  fmap _ Nil = Nil
  fmap f (Cons a as) = Cons (f a) (fmap f as)

instance Applicative List where
  pure :: a -> List a
  pure a = Cons a Nil
  (<*>) :: List (a -> b) -> List a -> List b
  Nil <*> _ = Nil
  _ <*> Nil = Nil
  fs <*> xs = flatMap (<$> xs) fs

instance Monad List where
  return = pure
  (>>=) xs f = flatMap f xs

instance Arbitrary a => Arbitrary (List a) where
  arbitrary = do
    hlist <- arbitrary
    let l = haskellListToList hlist
    return l

instance (Eq a) => EqProp (List a) where
  (=-=) = eq

haskellListToList :: [a] -> List a
haskellListToList = foldr Cons Nil

append :: List a -> List a -> List a
append Nil ys = ys
append xs Nil = xs
append (Cons x xs) ys = Cons x $ xs `append` ys

fold :: (a -> b -> b) -> b -> List a -> b
fold _ b Nil = b
fold f b (Cons h t) = f h (fold f b t)

concat' :: List (List a) -> List a
concat' = fold append Nil

flatMap :: (a -> List b) -> List a -> List b
flatMap f as = concat' $ fmap f as

### Exercise 2

Write the following functions using the methods provided by `Monad` and `Functor`. Using stuff like identity and composition is fine, but it has to typecheck with types provided.

1.
```haskell
j :: Monad m => m (m a) -> m a
```

Expecting the following behavior:
```
Prelude> j [[1, 2], [], [3]]
[1,2,3]
Prelude> j (Just (Just 1))
Just 1
Prelude> j (Just Nothing)
Nothing
Prelude> j Nothing
Nothing
```

In [1]:
j :: Monad m => m (m a) -> m a
j mma = mma >>= id

Line 2: Use join
Found:
mma >>= id
Why not:
Control.Monad.join mma

2.
```haskell
l1 :: Monad m => (a -> b) -> m a -> m b
```

In [4]:
l1 :: Monad m => (a -> b) -> m a -> m b
l1 = fmap

3.
```haskell
l2 :: Monad m
    => (a -> b -> c) -> m a -> m b -> m c
```

In [9]:
l2 :: Monad m => (a -> b -> c) -> m a -> m b -> m c
l2 f ma mb =
    ma >>= \a ->
    mb >>= \b ->
    return $ f a b

5. You’ll need recursion for this one.
```haskell
meh :: Monad m => [a] -> (a -> m b) -> m [b]
```

In [ ]:
meh :: Monad m => [a] -> (a -> m b) -> m [b]
meh (x:xs) f =
    f x >>= \y ->
    meh xs f >>= \ys ->
    return $ y:ys

6. Hint: reuse `meh`

```haskell
flipType :: (Monad m) => [m a] -> m [a]
```

In [ ]:
flipType :: (Monad m) => [m a] -> m [a]
flipType mas = meh mas id